# BIG WARNING

I'm still developing the interface for this, just trying to get Heroku and docker containers for it working before I clean things up. Do not use yet

In [1]:
%load_ext autoreload
%autoreload 2
import locale
import pandas as pd
import numpy as np
from rentorown.rentorown import RentOrOwn
import ipywidgets as widgets
from IPython.display import display
locale.setlocale(locale.LC_ALL, '');
pd.options.display.max_rows = 999

Set the rental characteristics

In [2]:
style = {'description_width': 'initial'}
rent_box = widgets.IntText(
    value=1000,
    description="Monthly Rent:",
    disabled=False,
    style=style
)
display(rent_box)

IntText(value=1000, description='Monthly Rent:', style=DescriptionStyle(description_width='initial'))

Set the ownership characteristics

In [3]:
house_price_box = widgets.IntText(
    value=100_000,
    description="House purchase price: ",
    disabled=False,
    style=style
)
display(house_price_box)

IntText(value=100000, description='House purchase price: ', style=DescriptionStyle(description_width='initial'…

Set the down payment, either use the slider to select a percentage, or directly input a dollar amount

In [4]:

down_payment_pct_box = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1.0,
    step=0.01,
    description="% down: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".0%",
    style=style
)
dp_dollar = widgets.Output()
display(down_payment_pct_box, dp_dollar)
def down_payment_event(btn_object):
    with dp_dollar:
        dp_dollar.clear_output()
        print(f"Down payment dollars: ${house_price_box.value * btn_object['new']:n}")


down_payment_pct_box.observe(down_payment_event, names="value")

FloatSlider(value=0.1, continuous_update=False, description='% down: ', max=1.0, readout_format='.0%', step=0.…

Output()

In [5]:
down_payment_box = widgets.IntText(
    value=20_000,
    description="Down payment in dollars",
    style=style
)
def dp_slider_change(change):
    down_payment_box.value = house_price_box.value * change.new

down_payment_pct_box.observe(dp_slider_change, names='value')
display(down_payment_box)

IntText(value=20000, description='Down payment in dollars', style=DescriptionStyle(description_width='initial'…

In [6]:
mortgage_amortization_slider = widgets.IntSlider(
    value=25,
    min=1,
    max=30,
    step=1,
    description="Years of amortization: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style=style
)
display(mortgage_amortization_slider)

IntSlider(value=25, continuous_update=False, description='Years of amortization: ', max=30, min=1, style=Slide…

In [7]:
mortgage_apr_slider = widgets.FloatSlider(
    value=0.03,
    min=0.0,
    max=0.3,
    step=0.001,
    description="Mortgage APR rate: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1%"
)
display(mortgage_apr_slider)

FloatSlider(value=0.03, continuous_update=False, description='Mortgage APR rate: ', max=0.3, readout_format='.…

In [8]:
calc_button = widgets.Button(description="Run Simulation")
out_widget = widgets.Output()
display(calc_button)
display(out_widget)
def calculate_rent_or_own(btn_object):
    with out_widget:
        out_widget.clear_output()
        mtg = RentOrOwn(
            monthly_rent=rent_box.value,
            house_price=house_price_box.value,
            down_payment=down_payment_box.value,
            mortgage_amortization_years=mortgage_amortization_slider.value,
            mortgage_apr=mortgage_apr_slider.value,
            housing_asset_dict={"dist": np.random.normal, "dist_args": {"loc": 0.004, "scale": 0.0136}},
            investment_asset_dict={"dist": np.random.normal, "dist_args": {"loc": 0.00510, "scale": 0.0266}},
            number_of_simulations=1_000,
            additional_purchase_costs=0,
            additional_monthly_costs=50,
            mortgage_payment_schedule="monthly",
            mortgage_additional_payments=0,
            annual_inflation=0.02,
            monthly_property_tax_rate=None,
            maintenance_cost=0.01
        )
        mtg.histogram()
        mtg.median_returns_plot()
        display(mtg.mortgage_df)

calc_button.on_click(calculate_rent_or_own)


Button(description='Run Simulation', style=ButtonStyle())

Output()